# Building Semantic Memory with Embeddings

In this notebook, we show how to use [Chroma](https://www.trychroma.com/) with Semantic Kernel to create even more
intelligent applications. We assume that you are already familiar with the concepts of Semantic Kernel
and memory. [Previously](04-context-variables-chat.ipynb), we have used `context variables` to pass
additional text into prompts, enriching them with more context for a basic chat experience.

However, relying solely on context variables has its limitations, such as the model's token limit.
To overcome these limitations, we will use **SK Semantic Memory**, leveraging Chroma as a persistent
Semantic Memory Storage.

**Chroma** is an open-source embedding database designed to make it easy to build Language Model
applications by making knowledge, facts, and skills pluggable for LLMs. It allows us to store and
retrieve information in a way that can be easily utilized by the models, enabling both short-term
and long-term memory for more advanced applications. In this notebook, we will showcase how to
effectively use Chroma with the Semantic Kernel for a powerful application experience.

In [1]:
#r "nuget: Microsoft.SemanticKernel, 0.17.230629.1-preview"
#r "nuget: Microsoft.SemanticKernel.Connectors.Memory.Chroma, 0.17.230629.1-preview"
#r "nuget: System.Linq.Async, 6.0.1"

#!import config/Settings.cs

using System;
using System.Collections.Generic;
using System.Linq;
using System.Threading.Tasks;
using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.Connectors.Memory.Chroma;
using Microsoft.SemanticKernel.Memory;

Installed Packages Microsoft.SemanticKernel, 0.17.230629.1-preview Microsoft.SemanticKernel.Connectors.Memory.Chroma, 0.17.230629.1-preview System.Linq.Async, 6.0.1

In order to use memory, we need to instantiate the Kernel with a Memory Storage
and an Embedding backend. In this example, we make use of the `ChromaMemoryStore`,
leveraging [Chroma](https://www.trychroma.com/), an open source embedding database
you can run locally and in the cloud.

To run Chroma locally, here's a quick script to download Chroma source and run it using Docker:

```shell
git clone https://github.com/chroma-core/chroma.git
cd chroma
docker-compose up --build
```

In [2]:
var (useAzureOpenAI, model, azureEndpoint, apiKey, orgId) = Settings.LoadFromFile();

var builder = new KernelBuilder();

if (useAzureOpenAI)
{
    builder.WithAzureTextEmbeddingGenerationService("text-embedding-ada-002", azureEndpoint, apiKey);
    builder.WithAzureTextCompletionService(model, azureEndpoint, apiKey);
}
else
{
    builder.WithOpenAITextEmbeddingGenerationService("text-embedding-ada-002", apiKey);
    builder.WithOpenAITextCompletionService(model, apiKey, orgId);
}

var chromaMemoryStore = new ChromaMemoryStore("http://127.0.0.1:8000");

builder.WithMemoryStorage(chromaMemoryStore);

var kernel = builder.Build();

At its core, Semantic Memory is a set of data structures that allows to store
the meaning of text that come from different data sources, and optionally to
store the source text and other metadata.

The text can be from the web, e-mail providers, chats, a database, or from your
local directory, and are hooked up to the Semantic Kernel through memory connectors.

The texts are embedded, sort of "compressed", into a vector of floats that representing
mathematically the text content and meaning.

You can read more about embeddings [here](https://aka.ms/sk/embeddings).

### Manually adding memories

Let's create some initial memories "About Me". We can add memories to `ChromaMemoryStore` by using `SaveInformationAsync`

In [5]:
const string MemoryCollectionName = "aboutMe";

await kernel.Memory.SaveInformationAsync(MemoryCollectionName, id: "info1", text: "My name is Andrea");
await kernel.Memory.SaveInformationAsync(MemoryCollectionName, id: "info2", text: "I currently work as a tourist operator");
await kernel.Memory.SaveInformationAsync(MemoryCollectionName, id: "info3", text: "I currently live in Seattle and have been living there since 2005");
await kernel.Memory.SaveInformationAsync(MemoryCollectionName, id: "info4", text: "I visited France and Italy five times since 2015");
await kernel.Memory.SaveInformationAsync(MemoryCollectionName, id: "info5", text: "My family is from New York");

Let's try searching the memory:

In [6]:
var questions = new[]
{
    "what is my name?",
    "where do I live?",
    "where is my family from?",
    "where have I travelled?",
    "what do I do for work?",
};

foreach (var q in questions)
{
    var response = await kernel.Memory.SearchAsync(MemoryCollectionName, q, limit: 1, minRelevanceScore: 0.5).FirstOrDefaultAsync();
    Console.WriteLine(q + " " + response?.Metadata.Text);
}

what is my name? My name is Andrea
where do I live? I currently live in Seattle and have been living there since 2005
where is my family from? My family is from New York
where have I travelled? I visited France and Italy five times since 2015
what do I do for work? I currently work as a tourist operator


Let's now revisit our chat sample from the [previous notebook](04-context-variables-chat.ipynb).
If you remember, we used context variables to fill the prompt with a `history` that continuously got populated as we chatted with the bot. Let's add also memory to it!

This is done by using the `TextMemorySkill` which exposes the `recall` native function.

`recall` takes an input ask and performs a similarity search on the contents that have
been embedded in the Memory Store. By default, `recall` returns the most relevant memory.

In [7]:
using Microsoft.SemanticKernel.Skills.Core;

// TextMemorySkill provides the "recall" function
kernel.ImportSkill(new TextMemorySkill());

In [8]:
const string skPrompt = @"
ChatBot can have a conversation with you about any topic.
It can give explicit instructions or say 'I don't know' if it does not have an answer.

Information about me, from previous conversations:
- {{$fact1}} {{recall $fact1}}
- {{$fact2}} {{recall $fact2}}
- {{$fact3}} {{recall $fact3}}
- {{$fact4}} {{recall $fact4}}
- {{$fact5}} {{recall $fact5}}

Chat:
{{$history}}
User: {{$userInput}}
ChatBot: ";

var chatFunction = kernel.CreateSemanticFunction(skPrompt, maxTokens: 200, temperature: 0.8);

The `RelevanceParam` is used in memory search and is a measure of the relevance score from 0.0 to 1.0, where 1.0 means a perfect match. We encourage users to experiment with different values.

In [9]:
var context = kernel.CreateNewContext();

context["fact1"] = "what is my name?";
context["fact2"] = "where do I live?";
context["fact3"] = "where is my family from?";
context["fact4"] = "where have I travelled?";
context["fact5"] = "what do I do for work?";

context[TextMemorySkill.CollectionParam] = MemoryCollectionName;
context[TextMemorySkill.RelevanceParam] = "0.6";

Now that we've included our memories, let's chat!

In [10]:
var history = "";
context["history"] = history;
Func<string, Task> Chat = async (string input) => {
    // Save new message in the context variables
    context["userInput"] = input;

    // Process the user message and get an answer
    var answer = await chatFunction.InvokeAsync(context);

    // Append the new interaction to the chat history
    history += $"\nUser: {input}\nChatBot: {answer}\n";
    context["history"] = history;
    
    // Show the bot response
    Console.WriteLine("ChatBot: " + context);
};

In [11]:
await Chat("Hello, I think we've met before, remember? my name is...");

ChatBot:  Hi there! Yes, I remember you. Your name is Andrea, right?


In [12]:
await Chat("I want to plan a trip and visit my family. Do you know where that is?");

ChatBot:  Yes, I remember you said your family is from New York. Is that where you want to go?


In [13]:
await Chat("Great! What are some fun things to do there?");

ChatBot:  There are lots of fun things to do in New York! You could visit the Empire State Building, take a boat ride around Manhattan, explore Central Park, or take a tour of the Statue of Liberty.


### Adding documents to your memory

Many times in your applications you'll want to bring in external documents into your memory. Let's see how we can do this using ChromaMemoryStore.

Let's first get some data using some of the links in the Semantic Kernel repo.

In [17]:
const string memoryCollectionName = "SKGitHub";

var githubFiles = new Dictionary<string, string>()
{
    ["https://github.com/microsoft/semantic-kernel/blob/main/README.md"]
        = "README: Installation, getting started, and how to contribute",
    ["https://github.com/microsoft/semantic-kernel/blob/main/samples/notebooks/dotnet/02-running-prompts-from-file.ipynb"]
        = "Jupyter notebook describing how to pass prompts from a file to a semantic skill or function",
    ["https://github.com/microsoft/semantic-kernel/blob/main/samples/notebooks/dotnet/00-getting-started.ipynb"]
        = "Jupyter notebook describing how to get started with the Semantic Kernel",
    ["https://github.com/microsoft/semantic-kernel/tree/main/prompts/samples/ChatPlugin/ChatGPT"]
        = "Sample demonstrating how to create a chat skill interfacing with ChatGPT",
    ["https://github.com/microsoft/semantic-kernel/blob/main/dotnet/src/SemanticKernel/Memory/Volatile/VolatileMemoryStore.cs"]
        = "C# class that defines a volatile embedding store",
    ["https://github.com/microsoft/semantic-kernel/tree/main/samples/dotnet/KernelHttpServer/README.md"]
        = "README: How to set up a Semantic Kernel Service API using Azure Function Runtime v4",
    ["https://github.com/microsoft/semantic-kernel/tree/main/samples/apps/chat-summary-webapp-react/README.md"]
        = "README: README associated with a sample starter react-based chat summary webapp",
};

Let's build a new Kernel.

In [18]:
var builder = new KernelBuilder();

if (useAzureOpenAI)
{
    builder.WithAzureTextEmbeddingGenerationService("text-embedding-ada-002", azureEndpoint, apiKey);
    builder.WithAzureTextCompletionService(model, azureEndpoint, apiKey);
}
else
{
    builder.WithOpenAITextEmbeddingGenerationService("text-embedding-ada-002", apiKey);
    builder.WithOpenAITextCompletionService(model, apiKey, orgId);
}

builder.WithMemoryStorage(new ChromaMemoryStore("http://127.0.0.1:8000"));

var kernel = builder.Build();

Now let's add these files to ChromaMemoryStore using `SaveReferenceAsync`.

In [19]:
Console.WriteLine("Adding some GitHub file URLs and their descriptions to Chroma Semantic Memory.");
var i = 0;
foreach (var entry in githubFiles)
{
    await kernel.Memory.SaveReferenceAsync(
        collection: memoryCollectionName,
        description: entry.Value,
        text: entry.Value,
        externalId: entry.Key,
        externalSourceName: "GitHub"
    );
    Console.WriteLine($"  URL {++i} saved");
}

Adding some GitHub file URLs and their descriptions to Chroma Semantic Memory.
  URL 1 saved
  URL 2 saved
  URL 3 saved
  URL 4 saved
  URL 5 saved
  URL 6 saved
  URL 7 saved


In [20]:
string ask = "I love Jupyter notebooks, how should I get started?";
Console.WriteLine("===========================\n" +
                    "Query: " + ask + "\n");

var memories = kernel.Memory.SearchAsync(memoryCollectionName, ask, limit: 5, minRelevanceScore: 0.6);

i = 0;
await foreach (MemoryQueryResult memory in memories)
{
    Console.WriteLine($"Result {++i}:");
    Console.WriteLine("  URL:     : " + memory.Metadata.Id);
    Console.WriteLine("  Title    : " + memory.Metadata.Description);
    Console.WriteLine("  Relevance: " + memory.Relevance);
    Console.WriteLine();
}

Query: I love Jupyter notebooks, how should I get started?

Result 1:
  URL:     : https://github.com/microsoft/semantic-kernel/blob/main/samples/notebooks/dotnet/00-getting-started.ipynb
  Title    : Jupyter notebook describing how to get started with the Semantic Kernel
  Relevance: 0.7357226014137268

Result 2:
  URL:     : https://github.com/microsoft/semantic-kernel/blob/main/samples/notebooks/dotnet/02-running-prompts-from-file.ipynb
  Title    : Jupyter notebook describing how to pass prompts from a file to a semantic skill or function
  Relevance: 0.6330309212207794

Result 3:
  URL:     : https://github.com/microsoft/semantic-kernel/blob/main/README.md
  Title    : README: Installation, getting started, and how to contribute
  Relevance: 0.6175204217433929

